In [1]:
import urllib
import requests
import pandas as pd
import json
import time

import module_api_raw_data as ma
import module_read_file as mr
import module_save_file as ms

In [5]:
# task1 extract
# 擷取全國年度的合併csv資料並轉為df
dir_path = "/workspaces/TIR104_g2/A0_raw_data/tw/tw_movie_2022-2025/"
file_name = "TWMovie2022-2025.csv"
file_path = dir_path + file_name
df_tw_annual = mr.read_file_to_df(file_path)

In [ ]:
# task2 transform
# 將Name欄位名稱去除()
# 將
df_tw_annual["Name"] = df_tw_annual["Name"].str.split("(").str[0]
# df_tw_annual = df_tw_annual.loc[:, ["Year", "MovieId", "Name"]]

In [8]:
# task3 extract
# 針對台灣總表取出 Name 欄位，並搜尋每個內容
# 存入查詢總結果list
query_list = df_tw_annual["Name"]
total_search_results = ma.tmdb_list_search_results(query_list)

關鍵字：PHANTOM: THE MUSICAL FILM，查無相關結果
關鍵字：遊走塵市間，查無相關結果
關鍵字：時間之舞，查無相關結果
關鍵字：黎巴嫩的天空，查無相關結果
關鍵字：自然就樹美，查無相關結果
關鍵字：逃出風暴，查無相關結果
關鍵字：騎趣環世界，查無相關結果
關鍵字：危雞總動員，查無相關結果
關鍵字：達賴喇嘛:大腦故事 ，查無相關結果
關鍵字：請為我在羅馬多留一天，查無相關結果
關鍵字：戰地夜曲，查無相關結果
關鍵字：黑貓白貓：4K數位修復版，查無相關結果
關鍵字：慕夏狂潮：從崛起到永恆，查無相關結果
關鍵字：凝弒，查無相關結果
關鍵字：愛在暹邏：數位經典版，查無相關結果
關鍵字：花樣年華 4K數位修復版，查無相關結果
關鍵字：重慶森林 4K數位修復版，查無相關結果
關鍵字：墮落天使  4K數位修復版，查無相關結果
關鍵字：愛在他鄉的季節，查無相關結果
關鍵字：遇上1967的女神，查無相關結果
關鍵字：末代皇帝（數位修復版），查無相關結果
關鍵字：伊恩•麥克連 80歲個人秀巡迴演出（英國國家劇院現場），查無相關結果
關鍵字：法蘭克札帕：從前衛到當代，查無相關結果
關鍵字：羅密歐與茱麗葉（英國國家劇院現場），查無相關結果
關鍵字：初步舉證，查無相關結果
關鍵字：利奥波德城（英國國家劇院現場），查無相關結果
關鍵字：丹尼爾約翰斯頓：魔鬼詩篇，查無相關結果
關鍵字：波西米亞人（英國皇家歌劇院），查無相關結果
關鍵字：馬克白（英國皇家歌劇院），查無相關結果
關鍵字：夢迴西藏：軍官與達賴喇嘛，查無相關結果
關鍵字：偷香 數位修復版，查無相關結果
關鍵字：曼儂‧雷斯科（英國皇家歌劇院），查無相關結果
關鍵字：兩情世界，查無相關結果
關鍵字：打擊驚魂，查無相關結果
關鍵字：蜘蛛人：無家日 戲院加長版，查無相關結果
關鍵字：探戈行不行，查無相關結果
關鍵字：投機者樂團：愛的鼓勵，查無相關結果
關鍵字：盜夢睡務員，查無相關結果
關鍵字：藝術恐怖份子 班克斯，查無相關結果
關鍵字：圓滾滾藝術家-費爾南多·波特羅，查無相關結果
關鍵字：馬達加斯加：狐猴之島，查無相關結果
關鍵字：最後的冰川，查無相關結果
關鍵字：不可侵犯：溫斯坦性醜聞，查無相關結果
關鍵字：哈勃望遠鏡，查無相關結果
關鍵字：小行星獵人，查無相關結果
關鍵字：回歸野性，查無相關結果
關鍵字：機遇之歌：數位修

In [ ]:
# task4 tansform
# 針對Name、title新增處理空白、大小寫的欄位
df_tmdb_search_results = pd.DataFrame(total_search_results)
df_tmdb_search_results["title_t"] = df_tmdb_search_results["title"].str.replace(" ", "").str.upper()
df_tw_annual["Name_t"] = df_tw_annual["Name"].str.replace(" ", "").str.upper()

In [20]:
# task5 tansform
# 用處理過的名稱欄位 merge df_tw_annual 跟 df_tmdb_search_results

df_mapping = df_tw_annual.merge(
    # 因為比對結果，會有重複電影名稱，僅保留第一筆
    df_tmdb_search_results.drop_duplicates(subset=["title"]),

    how="left",
    #df_tw_annual
    left_on="Name_t",
    #df_search_results
    right_on="title_t",
)

In [18]:
# task6 load
# 存原始資料進A0_raw_data
dir_path = "/workspaces/TIR104_g2/A0_raw_data/tw/tw_mapping_tmdb"
file_name = "v2_mapping_close_true.csv"
ms.save_as_csv(df_mapping, file_name, dir_path)

v2_mapping_close_true.csv 儲存成功, 存放路徑: /workspaces/TIR104_g2/A0_raw_data/tw/tw_mapping_tmdb/v2_mapping_close_true.csv


In [23]:
nan_count = df_mapping["id"].isna().sum()
success_count = df_mapping["id"].notna().sum()
print(f"比對判定失敗: {nan_count} 筆資料")
print(f"比對判定成功: {success_count} 筆資料")

比對判定失敗: 492 筆資料
比對判定成功: 1776 筆資料
